# Richmeister Integration Test Setup

This book allows the installation of test infrastructure, including tables in two regions and installation and configuration of the replication functions for multi-master replication of the tables.

## Asset Deployment

In [ ]:
# Deploy to the east region
import os
deploy_east = os.environ['DEPLOY_BUCKET_EAST']

In [ ]:
%%script env DEPLOY_BUCKET="$deploy_east" bash
cd ../../
make package
make deploy

In [ ]:
# Deploy to the west
deploy_west = os.environ['DEPLOY_BUCKET_WEST']

In [ ]:
%%script env DEPLOY_BUCKET="$deploy_west" bash
cd ../../
make package
make deploy

## Test Table Creation

In [ ]:
import boto3

ddb_east = boto3.client('dynamodb', region_name='us-east-1')
ddb_west = boto3.client('dynamodb', region_name='us-west-2')

table_name = 'PKTestTable'

In [ ]:
create_table_east_response = ddb_east.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'S'
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 20
    },
    StreamSpecification={
        'StreamEnabled': True,
        'StreamViewType': 'NEW_AND_OLD_IMAGES'
    }
)

print create_table_east_response

In [ ]:
table_east_stream = create_table_east_response['TableDescription']['LatestStreamArn']
print table_east_stream

In [ ]:
create_table_west_response = ddb_west.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'S'
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 20
    },
    StreamSpecification={
        'StreamEnabled': True,
        'StreamViewType': 'NEW_AND_OLD_IMAGES'
    }
)

print create_table_west_response

In [ ]:
table_west_stream = create_table_west_response['TableDescription']['LatestStreamArn']
print table_west_stream

## Install Replicator Stacks

In [ ]:
cf_east = boto3.client('cloudformation', region_name='us-east-1')
cf_west = boto3.client('cloudformation', region_name='us-west-2')

replicator_east_stack = 'replicatorEastPK'
replicator_west_stack = 'replicatorWestPK'

In [ ]:
# East
template_url = 'https://s3.amazonaws.com/' + deploy_east + '/replicator.yml'

stack_east = cf_east.create_stack(
    StackName=replicator_east_stack,
    TemplateURL=template_url,
    Parameters=[
        {
            'ParameterKey': 'TableName',
            'ParameterValue': table_name
        },
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': deploy_east
        },
        {
            'ParameterKey': 'StreamArn',
            'ParameterValue': table_east_stream
        },
        {
            'ParameterKey': 'DestinationTable',
            'ParameterValue': table_name
        },
        {
            'ParameterKey': 'DestinationRegion',
            'ParameterValue': 'us-west-2'
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print stack_east

In [ ]:
# West
template_url = 'https://s3-us-west-2.amazonaws.com/' + deploy_west + '/replicator.yml'

stack_west = cf_west.create_stack(
    StackName=replicator_west_stack,
    TemplateURL=template_url,
    Parameters=[
        {
            'ParameterKey': 'TableName',
            'ParameterValue': table_name
        },
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': deploy_west
        },
        {
            'ParameterKey': 'StreamArn',
            'ParameterValue': table_west_stream
        },
        {
            'ParameterKey': 'DestinationTable',
            'ParameterValue': table_name
        },
        {
            'ParameterKey': 'DestinationRegion',
            'ParameterValue': 'us-east-1'
        }       
        
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print stack_west

In [ ]:
# Wait for stack creation to complete

print 'waiting for east stack complete...'
waiter = cf_east.get_waiter('stack_create_complete')
waiter.wait(
    StackName=replicator_east_stack
)

print 'waiting for west stack complete...'
waiter = cf_west.get_waiter('stack_create_complete')
waiter.wait(
    StackName=replicator_west_stack
)

print 'stack create complete'

## Run Integration Tests

This assumes you have [radish](http://radish-bdd.io/) installed.

This also assumes you have executed a setenv prior to running this book that defines:

* AWS_DEFAULT_PROFILE
* AWS_DEFAULT_REGION
* https_proxy
* DEPLOY_BUCKET_EAST
* DEPLOY_BUCKET_WEST

If you ran the above set up ok then you should be good to go

In [ ]:
%%script bash
radish InsertReplication.feature

In [ ]:
%%script bash
radish ModifySetup.feature

In [ ]:
%%script bash
radish ModifyReplication.feature

In [ ]:
%%script bash
radish DeleteReplication.feature

## Utilities

### Invoking the Copy Lambda

Useful for troubleshooting, clean up stuff, messing around

In [ ]:
lambda_east = boto3.client('lambda')

response = lambda_east.invoke(
    FunctionName=east_copy_fn,
    LogType='Tail'
)

print response

### Get an Item

Useful for troubleshooting

In [ ]:
response = ddb_east.get_item(
    TableName=table_name,
    Key={
        "Id": {
            "S": 'b1'
        }
    }
)

print response
print 'Item' in response
item = response['Item']
remote_ts = item['ts']['N']
print remote_ts
remote_wid = item['wid']['S']
print remote_wid
print 'replicate' in item

## Clean Out Tables

In [ ]:
# Delete the items in the non-replicated region, and let the remote region replicate the delete to the primary
# region
response = ddb_west.scan(
    TableName=table_name
)

print response

if 'Items' in response:
    scanned_items = response['Items']
    print scanned_items

    for item in scanned_items:
        id = item['Id']
        response = ddb_west.delete_item(
            TableName=table_name,
            Key={
                'Id': id
            }
        )

        print response
else:
    print 'Nothing to delete'

In [ ]:
# Look for any remaining items in the east - may have some
# leftover data due to bugs, or data not replicated (as expected)
response = ddb_east.scan(
    TableName=table_name
)

print response

if 'Items' in response:
    scanned_items = response['Items']
    print scanned_items

    for item in scanned_items:
        id = item['Id']
        print item
        if 'replicate' in item:
            del item['replicate']
            print item

            response = ddb_east.put_item(
                TableName=table_name,
                Item=item
            )

            print response
else:
    print 'Nothing to delete'
    
response = ddb_east.scan(
    TableName=table_name
)

print response

if 'Items' in response:
    scanned_items = response['Items']
    print scanned_items

    for item in scanned_items:
        id = item['Id']
        response = ddb_east.delete_item(
            TableName=table_name,
            Key={
                'Id': id
            }
        )

        print response
else:
    print 'Nothing to delete'

## Clean Up

In [ ]:
print 'destroy {}'.format(replicator_east_stack)
response = cf_east.delete_stack(
    StackName=replicator_east_stack
)

print 'destroy {}'.format(replicator_west_stack)
response = cf_west.delete_stack(
    StackName=replicator_west_stack
)

print 'waiting for east stack delete...'
waiter = cf_east.get_waiter('stack_delete_complete')
waiter.wait(
    StackName=replicator_east_stack
)

print 'waiting for west stack delete...'
waiter = cf_west.get_waiter('stack_delete_complete')
waiter.wait(
    StackName=replicator_west_stack
)

print 'stack tear down complete'

In [ ]:
# Delete test tables
for client in [ddb_east, ddb_west]:
    response = client.delete_table(
        TableName = table_name
    )
    
    print response